<a href="https://colab.research.google.com/github/YuehMintTai/GAETest20210722/blob/main/%E4%B8%89%E7%B8%BD%E9%96%80%E8%A8%BA%E5%8A%A9%E6%89%8B8_0.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install flask_ngrok
!pip install line-bot-sdk
!pip install bs4
!pip install requests

In [1]:
from flask_ngrok import (run_with_ngrok)
from flask import (Flask, request, abort)
from linebot import (LineBotApi, WebhookHandler)
from linebot.exceptions import (InvalidSignatureError)
from linebot.models import *
from linebot.models.flex_message import (BubbleContainer, IconComponent)
from linebot.models.actions import URIAction
from bs4 import BeautifulSoup
import requests
import OPDAid8_SN
import re
import myBubbles
import myBubbles2
import myBubbles3

In [12]:
app=Flask(__name__)
run_with_ngrok(app)

line_bot_api=LineBotApi(OPDAid8_SN.myLineBotApi)
handler=WebhookHandler(OPDAid8_SN.myWebhookHandler)
myID=OPDAid8_SN.myUserID

@app.route('/')
def index():
  return '歡迎使用三總北投門診助手8.0'

@app.route('/callback',methods=['POST'])
def callback():
  signature=request.headers['X-Line-Signature']
  body=request.get_data(as_text=True)
  try:
    handler.handle(body,signature)
  except InvalidSignatureError:
    abort(400)
  return 'OK'

@handler.add(MessageEvent,message=TextMessage)
def handle_message(event):
  message=text=event.message.text
  '''
  ##line_bot_api.reply_message(event.reply_token,myBubbles3.myBubble_drug())
  line_bot_api.reply_message(event.reply_token,myBubbles3.myBubble_Drs(4))
  '''
  if re.match('門診時間',message):
    line_bot_api.reply_message(event.reply_token, FlexSendMessage(alt_text=message,contents=myBubbles.myBubble_OPD1()))
  elif re.match('費用查詢',message):
    line_bot_api.reply_message(event.reply_token, FlexSendMessage(alt_text=message,contents=myBubbles.myBubble_Fee()))
  elif re.match('重大公告',message):
    line_bot_api.reply_message(event.reply_token, FlexSendMessage(alt_text=message,contents=myBubbles.myBubble_News()))
  elif re.match('院長',message):
    line_bot_api.reply_message(event.reply_token, myBubbles2.myBubble_dir())
  elif re.match('副院長',message):
    line_bot_api.reply_message(event.reply_token, myBubbles2.myBubble_vdir())
  elif re.match('醫療部主任',message):
    line_bot_api.reply_message(event.reply_token, myBubbles2.myBubble_head())
  elif re.match('藥癮科',message):
    line_bot_api.reply_message(event.reply_token, myBubbles3.myBubble_Drs(0))
  elif re.match('心身精神科',message):
    line_bot_api.reply_message(event.reply_token, myBubbles3.myBubble_Drs(1))
  elif re.match('老人精神科',message):
    line_bot_api.reply_message(event.reply_token, myBubbles3.myBubble_Drs(2))
  elif re.match('神經精神科',message):
    line_bot_api.reply_message(event.reply_token, myBubbles3.myBubble_Drs(3))
  elif re.match('軍陣精神科',message):
    line_bot_api.reply_message(event.reply_token, myBubbles3.myBubble_Drs(4))
  elif re.match('兒青科',message):
    line_bot_api.reply_message(event.reply_token, myBubbles3.myBubble_Drs(7))
  elif re.match('社區精神科',message):
    line_bot_api.reply_message(event.reply_token, myBubbles3.myBubble_Drs(6))
  elif re.match('家醫科/牙科',message):
    line_bot_api.reply_message(event.reply_token, myBubbles3.myBubble_Drs(8))
  elif re.match('門診兼任醫師',message):
    line_bot_api.reply_message(event.reply_token, myBubbles3.myBubble_Drs(9))
  elif re.match('成人精神科',message):
    line_bot_api.reply_message(event.reply_token, myBubbles3.myBubble_Drs(5))
  else:
    line_bot_api.push_message(myID,FlexSendMessage(alt_text='你好',contents=myBubbles.myBubble_greeting()))

app.run()

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)


 * Running on http://9a74-35-188-244-82.ngrok.io
 * Traffic stats available on http://127.0.0.1:4040


127.0.0.1 - - [03/Oct/2021 08:11:58] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [03/Oct/2021 08:11:58] "GET /favicon.ico HTTP/1.1" 404 -
127.0.0.1 - - [03/Oct/2021 08:12:34] "POST /callback HTTP/1.1" 200 -
127.0.0.1 - - [03/Oct/2021 08:12:46] "POST /callback HTTP/1.1" 200 -


In [2]:
##myBubble_dir出問題,要修正它...
app=Flask(__name__)
run_with_ngrok(app)

line_bot_api=LineBotApi(OPDAid8_SN.myLineBotApi)
handler=WebhookHandler(OPDAid8_SN.myWebhookHandler)
myID=OPDAid8_SN.myUserID

def myBubble_Drs(k):
  DrsContent=[]
  target_url='https://wwwv.tsgh.ndmctsgh.edu.tw/Doclist/194/30000/25075' 
  responses=requests.get(target_url)
  soup=BeautifulSoup(responses.text,'html.parser')
  myDivs=soup.find_all('div',{'class':'col-xs-12 col-md-12'})
  myTitle=myDivs[k].find('h3').contents[0].strip() ##i科
  for i, tag in enumerate(myDivs[k].find_all('a')):
      myButton=ButtonComponent(style='link',height='sm',
      action=URIAction(label=tag.contents[0].strip(),uri='https://wwwv.tsgh.ndmctsgh.edu.tw/'+tag.get('href')))
      DrsContent.append(myButton)
  myBubble_Drs=FlexSendMessage(
      alt_text=myTitle,
      contents=BubbleContainer(
        direction='ltr',
        hero=ImageComponent(
            url='https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcSblvaiDh5qhGVMEjT-5H6VDWuezc4mXvtogQ&usqp=CAU',
            size='full',
            aspect_ratio='20:13',
            aspect_mode='fit',
            action=actions.URIAction(uri='https://wwwv.tsgh.ndmctsgh.edu.tw/Doclist/194/30000/25075',label='醫師查詢')
        ),
        body=BoxComponent(
            layout='vertical',
            contents=[TextComponent(text=myTitle+'醫師查詢',weight='bold',size='lg',align='center')]),
        footer=BoxComponent(
            layout='vertical',spacing='sm',
            contents=DrsContent
        )
        ))
  return myBubble_Drs


@app.route('/')
def index():
  return '歡迎使用三總北投門診助手8.0'

@app.route('/callback',methods=['POST'])
def callback():
  signature=request.headers['X-Line-Signature']
  body=request.get_data(as_text=True)
  try:
    handler.handle(body,signature)
  except InvalidSignatureError:
    abort(400)
  return 'OK'

@handler.add(MessageEvent,message=TextMessage)
def handle_message(event):
  message=text=event.message.text
  ##line_bot_api.reply_message(event.reply_token,myBubbles3.myBubble_drug())
  line_bot_api.reply_message(event.reply_token,myBubbles3.myBubble_Drs(9))


  '''
  if re.match('門診時間',message):
    line_bot_api.reply_message(event.reply_token, FlexSendMessage(alt_text=message,contents=myBubbles.myBubble_OPD1()))
  elif re.match('費用查詢',message):
    line_bot_api.reply_message(event.reply_token, FlexSendMessage(alt_text=message,contents=myBubbles.myBubble_Fee()))
  elif re.match('重大公告',message):
    line_bot_api.reply_message(event.reply_token, FlexSendMessage(alt_text=message,contents=myBubbles.myBubble_News()))
  elif re.match('院長',message):
    line_bot_api.reply_message(event.reply_token, myBubbles2.myBubble_dir())
  elif re.match('副院長',message):
    line_bot_api.reply_message(event.reply_token, myBubbles2.myBubble_vdir())
  elif re.match('醫療部主任',message):
    line_bot_api.reply_message(event.reply_token, myBubbles2.myBubble_head())
  elif re.match('藥癮科',message):
    line_bot_api.reply_message(event.reply_token, myBubbles2.myBubble_all())###drug())
  elif re.match('心身精神科',message):
    line_bot_api.reply_message(event.reply_token, myBubbles2.myBubble_all())###mind())
  elif re.match('老人精神科',message):
    line_bot_api.reply_message(event.reply_token, myBubbles2.myBubble_all())###old())
  elif re.match('神經精神科',message):
    line_bot_api.reply_message(event.reply_token, myBubbles2.myBubble_all())###neuro())
  elif re.match('軍陣精神科',message):
    line_bot_api.reply_message(event.reply_token, myBubbles2.myBubble_all())###military())
  elif re.match('兒青科',message):
    line_bot_api.reply_message(event.reply_token, myBubbles2.myBubble_all())###child())
  elif re.match('社區精神科',message):
    line_bot_api.reply_message(event.reply_token, myBubbles2.myBubble_all())###ocial())
  elif re.match('家醫科/牙科',message):
    line_bot_api.reply_message(event.reply_token, myBubbles2.myBubble_all())###family())
  elif re.match('門診兼任醫師',message):
    line_bot_api.reply_message(event.reply_token, myBubbles2.myBubble_all())###others())
  else:
    line_bot_api.push_message(myID,FlexSendMessage(alt_text='你好',contents=myBubbles.myBubble_greeting()))
'''
app.run()

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)


 * Running on http://62bd-35-188-244-82.ngrok.io
 * Traffic stats available on http://127.0.0.1:4040


127.0.0.1 - - [03/Oct/2021 07:55:56] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [03/Oct/2021 07:55:56] "GET /favicon.ico HTTP/1.1" 404 -
127.0.0.1 - - [03/Oct/2021 07:56:22] "POST /callback HTTP/1.1" 200 -
